# Server Module

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [sse-results](https://github.com/Logiqx/sse-results) and is distributed under the terms of the GNU General Public License.

sse-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import time

from datetime import date, datetime, timedelta

import csv
import json

import logging
import threading
import queue

import socket
from urllib.request import urlopen

from common import Printable, projdir

from constants import *

## Generic Worker Thread and Thread Pool

See https://logiqx.github.io/sse-results/tech/motion.html for further details about the thread model.

In [2]:
# Default queue timeout dictates how quickly the worker thread can process the stop event
DEFAULT_QUEUE_TIMEOUT = 5

# Default worker timeout applies to things like socket connections, if not present in server.json 
DEFAULT_WORKER_TIMEOUT = 10

# Default loop interval determines the minimum time between each run of the manager thread 
DEFAULT_MANAGER_INTERVAL = 10

# Default manager threads dictates the number of workers 
DEFAULT_MANAGER_THREADS = 4

In [3]:
class WorkerThread():
    """Generic class for worker threads"""

    def __init__(self, config, motions, stopEvent, inQueue, outQueue, instance=None):
        """Simple initialisation"""

        self.name = self.__class__.__name__

        self.config = config
        self.motions = motions
        self.stopEvent = stopEvent
        self.inQueue = inQueue
        self.outQueue = outQueue
        self.instance = instance

        try: 
            self.timeout = self.config['Threading'][self.name]['Timeout']
        except KeyError:
            self.timeout = DEFAULT_WORKER_TIMEOUT

        self.thread = threading.Thread(target=self.main)


    def start(self):
        """Start the thread"""

        if self.instance:
            logging.debug("%s #%d starting", self.name, self.instance)
        else:
            logging.debug("%s starting", self.name)

        self.thread.start()


    def main(self):
        """Do the actual processing, until the stop event is detected"""

        while not self.stopEvent.is_set():
            motion = None

            try:
                motion = self.inQueue.get(block=True, timeout=DEFAULT_QUEUE_TIMEOUT)

                try:
                    self.process(motion)
                    
                except Exception as e:
                    # Setting the Motion status to unknown will ensure a retry, starting with detection
                    motion.status = HOST_STATUS_UNKNOWN
                    
                    # Let the manager thread know the queue item has been consumed
                    self.inQueue.task_done()
                    raise
                    
                else:
                    # Let the manager thread know the queue item has been consumed
                    self.inQueue.task_done()

            except queue.Empty:
                # Everything is fine but check for the stop event before retrying the queue
                pass
            
            except Exception as e:
                logging.error("Exception in %s - %s", self.name, e)


    def process(self, motion):
        """Default behavior is to warn that the class does not have a process() method"""
        
        logging.warning("%s does not have a process() method", self.name)


    def join(self):
        """Join the thread, essentially waiting for it to stop gracefully"""

        self.thread.join()

        if self.instance:
            logging.debug("%s #%d finished", self.name, self.instance)
        else:
            logging.debug("%s finished", self.name)

In [4]:
class WorkerThreadPool(WorkerThread):
    """Generic class for a worker thread pool, potentially consisting of a manager thread and multiple worker threads"""

    def __init__(self, config, motions, stopEvent, inQueue, outQueue, workerClass):
        """Simple initialisation"""
        
        super().__init__(config, motions, stopEvent, inQueue, outQueue)
        
        try: 
            self.interval = self.config['Threading'][self.name]['Interval']
        except KeyError:
            self.interval = DEFAULT_MANAGER_INTERVAL

        try: 
            self.threads = self.config['Threading'][self.name]['Threads']
        except KeyError:
            self.threads = DEFAULT_MANAGER_THREADS

        self.workers = []
        for i in range(self.threads):
            worker = workerClass(config, motions, stopEvent, inQueue, outQueue, i + 1)
            self.workers.append(worker)


    def start(self):
        """Start the worker threads"""

        super().start()
        
        for worker in self.workers:
            worker.start()
        

    def main(self):
        """Do the actual processing, until the stop event is detected"""

        try:
            while not self.stopEvent.is_set():
                # The manager must declare how many motions are to be processed 
                motionCount = self.process()

                # This wait ensures the thread pool manager runs no more frequently than every X seconds
                self.stopEvent.wait(self.interval)

                # Generate warning if the workers are not keeping up!
                if not self.stopEvent.is_set() and motionCount > 0 and not self.inQueue.empty():
                    logging.warning("%s has exceeded the interval time", self.name)

                # Wait for the queue to empty
                while not self.stopEvent.is_set() and not self.inQueue.empty():
                    self.stopEvent.wait(1)

        except Exception as e:
            logging.error("Exception in %s - %s", self.name, e)


    def process(self):
        """Default behavior is to warn that the class does not have a process() method"""
        
        logging.warning("%s does not have a process() method", self.name)
  

    def join(self):
        """Join the worker threads, ensuring they have all stopped cleanly"""
        
        super().join()
        
        for worker in self.workers:
            worker.join()

## Detection Thread Pool

Detect new motions appearing on the network

In [5]:
class DetectionThread(WorkerThread):
    """Thread pool worker to detect a motion appearing on the network"""

    def process(self, motion):
        """Do the actual processing of a single item"""

        if motion.status <= 0:
            logging.debug("%s checking network status of motion %i", self.name, motion.identifier)
        
            addrInfo = motion.getAddrInfo()
            
            if addrInfo:
                logging.debug("%s found motion %i at %s", self.name, motion.identifier, addrInfo[0])

                status = motion.getStatus(timeout=self.timeout)
                
                if status == HOST_STATUS_CONNECTED:
                    logging.info("%s confirmed motion %i is accepting connections", self.name, motion.identifier)
        
                    self.outQueue.put(motion)

                else:
                    logging.debug("%s confirmed motion %i is not accepting connections - status %i", self.name, motion.identifier, status)
            
            else:
                logging.debug("%s confirmed motion %i address information is unavailable", self.name, motion.identifier)

In [6]:
class DetectionThreadPool(WorkerThreadPool):
    """Thread pool manager to detect motions appearing on the network"""

    def __init__(self, config, motions, stopEvent, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(config, motions, stopEvent, inQueue, outQueue, DetectionThread)
        

    def process(self):
        """The thread pool manager puts motions onto the in queue for workers to consume"""
        
        logging.debug("%s queuing motions", self.name)
        
        count = 0
        for motion in self.motions.values():
            if motion.status <= 0:
                logging.debug("%s queuing motion %i", self.name, motion.identifier)
                self.inQueue.put(motion)
                count += 1
                
        return count

## Motion Inspection Thread

Inspect new motions appearing on the network

In [7]:
class InspectionThread(WorkerThread):
    """Thread pool worker to inspect a motion appearing on the network"""

    def process(self, motion):
        """Do the actual inspection of a single motion"""
        
        logging.debug("%s processing motion %i", self.name, motion.identifier)

        self.motion = motion

        self.fetchJson('info.json')
        self.fetchJson('settings.json')
        self.fetchJson('logs.json')
        
        # Change to HOST_STATUS_INSPECTED when downloads are implemented
        motion.status = HOST_STATUS_COMPLETED
        
        logging.debug("%s processed motion %i", self.name, motion.identifier)

        #self.outQueue.put(motion)


    def fetchJson(self, filename):
        """Do the actual inspection of a single motion"""
        
        motion = self.motion

        try:
            motion.fetchJson(filename, timeout=self.timeout)
        except Exception as e:
            logging.error("%s motion %i unable to return %s", self.name, motion.identifier, filename)
            raise

In [8]:
class InspectionThreadPool(WorkerThreadPool):
    """Thread pool manager to inspect motions appearing on the network"""

    def __init__(self, config, motions, stopEvent, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(config, motions, stopEvent, inQueue, outQueue, InspectionThread)


    def main(self):
        """There is no processing to be done in an unmanaged pool"""

        pass

## Download Thread Pool

Download new tracks for Motions

In [9]:
class DownloadThread(WorkerThread):
    """Thread pool worker to download tracks from a motion"""

    def process(self, motion):
        """Do the actual downloads for a single motion"""
        
        logging.debug("%s downloading from motion %i", self.name, motion.identifier)
        
        self.outQueue.put(motion)

In [10]:
class DownloadThreadPool(WorkerThreadPool):
    """Thread pool manager to download tracks from motions"""

    def __init__(self, config, motions, stopEvent, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(config, motions, stopEvent, inQueue, outQueue, DownloadThread)


    def main(self):
        """There is no processing to be done in an unmanaged pool"""

        pass

## Disconnection Thread Pool

Detect motions disconnecting from the network

In [11]:
class DisconnectionThread(WorkerThread):
    """Thread pool worker to detect a motion disconnected from the network"""

    def process(self, motion):
        """Do the actual processing of a single item"""
        
        logging.debug("%s checking network status of motion %i", self.name, motion.identifier)
        
        status = motion.getStatus(timeout=self.timeout)

        if motion.status > 0:
            logging.debug("%s confirmed motion %i is still connected", self.name, motion.identifier)
            
            # Note that motion.getStatus() will have nuked the status, thus need to set back to HOST_STATUS_COMPLETED
            motion.status = HOST_STATUS_COMPLETED

        else:
            logging.info("%s confirmed motion %i has been disconnected", self.name, motion.identifier)

In [12]:
class DisconnectionThreadPool(WorkerThreadPool):
    """Thread pool manager to detect motions disconnected from the network"""

    def __init__(self, config, motions, stopEvent, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(config, motions, stopEvent, inQueue, outQueue, DisconnectionThread)


    def process(self):
        """The thread pool manager puts motions onto the in queue for workers to consume"""
        
        logging.debug("%s queuing motions", self.name)
        
        count = 0
        for motion in self.motions.values():
            if motion.status == HOST_STATUS_COMPLETED:
                logging.debug("%s queuing motion %i", self.name, motion.identifier)
                self.inQueue.put(motion)
                count +=1
            
        return count

## Motion Class

Stuff relating to a single Motion device

In [13]:
HOST_STATUS_UNKNOWN = 0

HOST_STATUS_CONNECTED = 1
HOST_STATUS_INSPECTED = 2
HOST_STATUS_COMPLETED = 3

HOST_STATUS_REFUSED = -1
HOST_STATUS_TIMEOUT = -2
HOST_STATUS_GAIERROR = -3

DEFAULT_HTTP_PORT=80

DEFAULT_PING_TIMEOUT=5
DEFAULT_JSON_TIMEOUT=10

In [14]:
class Motion():
    '''Motion GPS'''
    
    def __init__(self, identifier):
        """Simple init method"""

        self.identifier = identifier

        self.hostname = f'motion-{identifier}'

        self.addrInfo = None
        self.status = HOST_STATUS_UNKNOWN

        self.config = {}
        self.json = {}
        
        
    def getAddrInfo(self, port=DEFAULT_HTTP_PORT):
        """Get address information (IP address) for Motion"""

        try:
            addrInfo = socket.getaddrinfo(self.hostname, port, proto=socket.IPPROTO_TCP)[0][4]

            # Sky SR101 router will sometimes return 10.10.10.10 and should be treated the same as socket.gaierror
            if addrInfo[0] == '10.10.10.10':
                raise socket.gaierror

        # Only really expect a socket.gaierror exception, either from socket.getaddrinfo or because of 10.10.10.10
        except socket.gaierror:
            self.addrInfo = None

        # Just in case of some other (unexpected) issue!
        except Exception:
            self.addrInfo = None

        # The IP address of the Motion has been successfully identified. Yay!
        else:
            self.addrInfo = addrInfo
            
        return self.addrInfo


    def getStatus(self, timeout=DEFAULT_PING_TIMEOUT):
        """Check the status of the Motion.

        Note the use of self.addrInfo instead of using the hostname with s.connect((host, port)).

        This avoids erroneous 'gaierror: [Errno -5] No address associated with hostname' under certain circumstances."""

        # Ensure address information is available
        if self.addrInfo is None:
            self.getAddrInfo()
        
        # Status can only be checked if address information is available
        if self.addrInfo:
            try:
                # Attempt connection to the standard HTTP socket as determined by socket.getaddrinfo()
                s = socket.socket()
                s.settimeout(timeout)
                s.connect(self.addrInfo)
                
                # Close the connection in a timely fashion - good practice
                s.shutdown(socket.SHUT_RDWR)
                s.close()
                
            # This should only happen if the IP address does not belong to a Motion
            except ConnectionRefusedError:
                self.status = HOST_STATUS_REFUSED

            # This should only happen when the Motion has been disconnected from the WiFi
            except socket.timeout:
                self.status = HOST_STATUS_TIMEOUT

            # Just in case of some other (unexpected) issue!
            except Exception:
                self.status = HOST_STATUS_UNKNOWN

            # The Motion is accepting connections. Yay!
            else:
                self.status = HOST_STATUS_CONNECTED

        # Address information is not known
        else:
            self.status = HOST_STATUS_GAIERROR

        return self.status
   

    def fetchJson(self, path, port=DEFAULT_HTTP_PORT, timeout=DEFAULT_JSON_TIMEOUT):
        """Fetch JSON from Motion"""

        result = None

        # Ensure address information is available
        if self.addrInfo is None:
            self.getAddrInfo()

        # JSON can only be fetched if address information is available
        if self.addrInfo:
            # Use addrInfo to avoid unnecessary DNS lookup which can take several seconds
            url = 'http://{}/{}'.format(self.addrInfo[0], path)

            # This is a simple but effective way to open a URL in Python
            response = urlopen(url, timeout=timeout)
            txt = response.read()

            # Parse the JSON and store it in a dictionary
            result = json.loads(txt)
            self.json[path] = result

        return result


    def summarise(self):
        """Print summary of device"""

        #print(self.hostname)
        #print()

        info = self.json['info.json']

        #print('Username = {}'.format(info['motion']['username']))
        print('Identifier = {}'.format(info['motion']['identifier']))
        #print()

        hardware = info['motion']['hardware']
        firmware = info['motion']['firmware']

        #print('Hardware = {}'.format(hardware))
        print('Firmware = {}{}'.format(firmware, ' (latest)' if firmware == LATEST_FIRMWARE else ''))
        #print()

        # 'filesystem level' always appears to be 3

        level = info['motion']['battery']['level']
        charging = info['motion']['battery']['charging']

        print('Battery = {}%'.format(level))
        print('Battery is {}charging'.format('not ' if charging == False else ''))
        #print()

        settings = self.json['settings.json']

        print('GNSS rate = {}'.format(settings['settings']['gnss_rate']))
        print()

        # Note: 'units': {'speed': 'kn', 'distance': 'nm'} does not impact logs (mm/s and mm)
        #        'start_date' is in UTC
        #        'timezone' and 'units' are applied by the built-in webserver
        #        'mode' always appears to be 0

        #logs = fetchJson(addresses[i][0], 'logs.json', timeout=20)
        #for log in reversed(logs['logs'][-1:]):
            #print(log)
        #print()

## Main Function

Startup and shutdown of all threads

In [15]:
def loadConfig():
    '''Read server config from JSON'''

    filename = os.path.join(projdir, CONFIG_DIR, 'server.json')
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        config = json.loads(jsonTxt)

    return config

In [16]:
def initLogging(level=logging.INFO, filename=None):
    """Initialise logging"""
    
    formatString = "%(asctime)s,%(levelname)s,%(message)s"
    dateTimeFormat = "%Y-%m-%d %H:%M:%S"

    # TODO - implement filename
    logging.basicConfig(format=formatString, datefmt=dateTimeFormat, level=level)

In [17]:
def dateRange(startDate, endDate):
    '''Determine range of dates'''

    dates = []

    for i in range(int((endDate - startDate).days) + 1):
        dates.append(startDate + timedelta(days = i))
        
    return dates


def readMotions():
    '''Read Motions from CSV file'''
    
    motions = {}

    year = datetime.now().year
    csvPath = os.path.join(projdir, EVENTS_DIR, str(year), CONFIG_DIR, 'motions.csv')

    with open(csvPath, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for motionUser in csvReader:
            motionId = int(motionUser['Motion ID'])

            startDate = datetime.strptime(motionUser['Start Date'], '%Y-%m-%d')
            endDate = datetime.strptime(motionUser['End Date'], '%Y-%m-%d')
            
            if motionId not in motions:
                motion = Motion(motionId)              
                motions[motionId] = motion
            else:
                motion = motions[motionId]

            for sessionDate in dateRange(startDate, endDate):
                sessionDate = sessionDate.strftime('%Y-%m-%d')

                # Note: Motion ID of zero (or negative) is a dummy record in the config
                if motionId > 0:
                    if sessionDate in motion.config:
                        logging.warning("Motion %i is assigned to multiple people on %s", motionId, sessionDate)
                    else:
                        motion.config[sessionDate] = motionUser
     
    return motions

In [18]:
def createThreadPools(config, motions, stopEvent):
    """Create the thread pools, including message queues"""

    threadPools = []

    # Detection   
    detectionInQueue = queue.Queue()
    detectionOutQueue = queue.Queue()
    detectionThreadPool = DetectionThreadPool(config, motions, stopEvent, detectionInQueue, detectionOutQueue)
    threadPools.append(detectionThreadPool)

    # Inspection    
    inspectionInQueue = detectionOutQueue
    inspectionOutQueue = queue.Queue()  
    inspectionThreadPool = InspectionThreadPool(config, motions, stopEvent, inspectionInQueue, inspectionOutQueue)
    threadPools.append(inspectionThreadPool)

    # Download
    downloadInQueue = inspectionOutQueue
    downloadOutQueue = queue.Queue()
    downloadThreadPool = DownloadThreadPool(config, motions, stopEvent, downloadInQueue, downloadOutQueue)
    threadPools.append(downloadThreadPool)

    # Disconnect
    disconnectionInQueue = queue.Queue()
    disconnectionOutQueue = queue.Queue()
    disconnectionThreadPool = DisconnectionThreadPool(config, motions, stopEvent, disconnectionInQueue, disconnectionOutQueue)
    threadPools.append(disconnectionThreadPool)

    return threadPools

In [19]:
def main():
    """Main function to start all of the threads"""

    # Read config
    config = loadConfig()

    # Initialise logging module
    initLogging(level=config['Logging']['Level'])
    
    # Read Motion data
    motions = readMotions()

    # Global event so that all threads can exit gracefully
    stopEvent = threading.Event()
    
    # Create the thread pools, including the message queues, etc.
    logging.info("Creating thread pools")
    threadPools = createThreadPools(config, motions, stopEvent)

    # Start up all of the thread pools
    abort = True
    try:
        logging.info("Starting thread pools")

        for threadPool in threadPools:
            threadPool.start()

        logging.info("Thread pools started")

        abort = False
    
    except Exception:
        logging.error("Thread pools failed to start")

    # Just sleep until a keyboard interrupt occurs
    while not abort:
        try:
            time.sleep(10)
        
        # Intercept keyboard interrupts to ensure that all threads are properly stopped, prior to exit
        except KeyboardInterrupt:
            logging.info("Keyboard interrupt")
            abort = True

    # Stop threads gracefully
    logging.info("Stopping thread pools")
    stopEvent.set()
    for threadPool in threadPools:
        threadPool.join()
    logging.info("Thread pools stopped")


if __name__ == '__main__':
    main()

2022-10-05 05:34:48,INFO,Creating thread pools
2022-10-05 05:34:48,INFO,Starting thread pools
2022-10-05 05:34:48,INFO,Thread pools started
2022-10-05 05:34:51,INFO,Keyboard interrupt
2022-10-05 05:34:51,INFO,Stopping thread pools
2022-10-05 05:34:58,INFO,Thread pools stopped
